In [1]:
import csv
import subprocess
from urllib.parse import urlparse

from dateutil.parser import parse as parse_datetime
from sqlalchemy import create_engine
import pandas as pd

from helpers import cleanup_dollar_value

In [2]:
TARGET_COLUMNS = [
    'hospital_id',
    #'row_id',
    'line_type',
    'description',
    'rev_code',
    'local_code',
    'code',
    'ms_drg',
    'apr_drg',
    'eapg',
    'hcpcs_cpt',
    'modifiers',
    'alt_hcpcs_cpt',
    'thru',
    'apc',
    'icd',
    'ndc',
    'drug_hcpcs_multiplier',
    'drug_quantity',
    'drug_unit_of_measurement',
    'drug_type_of_measurement',
    'billing_class',
    'setting',
    'payer_category',
    'payer_name',
    'plan_name',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_generic_notes',
    'additional_payer_specific_notes'
]

In [3]:
ccn = "230020"
url = "https://www.beaumont.org/docs/default-source/default-document-library/cdm-documents/2023/381405141_beaumont-hospital-dearborn-professional_standardcharges.csv"
filename = url.split("/")[-1].replace("%20", " ")
ein = filename.split("_")[0]
ein = ein[:2] + "-" + ein[2:]

In [4]:
import subprocess
subprocess.run(["wget", "--no-clobber", url, "-O", filename])

File ‘381405141_beaumont-hospital-dearborn-professional_standardcharges.csv’ already there; not retrieving.


CompletedProcess(args=['wget', '--no-clobber', 'https://www.beaumont.org/docs/default-source/default-document-library/cdm-documents/2023/381405141_beaumont-hospital-dearborn-professional_standardcharges.csv', '-O', '381405141_beaumont-hospital-dearborn-professional_standardcharges.csv'], returncode=1)

In [5]:
df_in = pd.read_csv(filename, encoding='latin-1', dtype={'Rev Code': str, 'Code': str}, low_memory=False)
df_in

,FS ID,CPT/HCPCS Code,Beaumont Health Charge Code Description,NDC,GROSS CHARGE,DEIDENTIFIED MIN,DEIDENTIFIED MAX,CASH PRICE,Aetna,Cofinity North Hospitals,...,Priority,UHC (Options PPO Beaumont),UHC (Options PPO Oakwood),UHC (Options PPO Botsford),UHC (All Payer Beaumont),UHC (All Payer Oakwood),UHC (All Payer Botsford),Min Price,Max Price,Unnamed: 26
0,Office Clinic Provider Fee,10009,PR FINE NEEDLE ASPIRATION BX W/CT GDN 1ST LESION,NaN,$618.00,$476.84,$840.62,$618.00,$513,$513.16,...,$561.76,$840.62,$596.00,$578.65,$782.53,$570.09,$553.49,$476.84,$840.62,NaN
1,Office Clinic Provider Fee,10021,PR FINE NEEDLE ASPIRATION BX W/O IMG GDN 1ST L...,NaN,$184.00,$106.12,$218.11,$184.00,$114,$113.63,...,$124.39,$218.11,$159.87,$155.22,$203.03,$152.92,$148.47,$106.12,$218.11,NaN
2,Office Clinic Provider Fee,10021,FNA W/O IMAGING GUIDANCE,NaN,$184.00,$106.12,$218.11,$184.00,$114,$113.63,...,$124.39,$218.11,$159.87,$155.22,$203.03,$152.92,$148.47,$106.12,$218.11,NaN
3,Office Clinic Provider Fee,10030,PR IMAGE-GUIDED CATHETER FLUID COLLECTION DRAI...,NaN,$443.00,$443.00,"$1,394.60",$443.00,$720,$719.65,...,$787.81,"$1,394.60",$990.04,$961.20,"$1,298.22",$947.00,$919.41,$685.38,"$1,394.60",NaN
4,Office Clinic Provider Fee,10030,GUIDE CATH FLUID DRAINAGE,NaN,$443.00,$443.00,"$1,394.60",$443.00,$720,$719.65,...,$787.81,"$1,394.60",$990.04,$961.20,"$1,298.22",$947.00,$919.41,$685.38,"$1,394.60",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24193,Hospital Outpatient Department Provider Fee,J0885,"EPOETIN ALFA, NON-ESRD",55513-0144-10,$18,$6.24,$18.00,$18.00,$9,$8.78,...,NaN,$9.78,$9.61,$9.61,$9.78,$9.61,$9.61,$6.24,$10.90,NaN
24194,Hospital Outpatient Department Provider Fee,J0894,PR DECITABINE INJECTION,67457-0316-25,$58,$0.20,$58.00,$58.00,$3,$2.93,...,NaN,$4.12,$4.05,$4.05,$4.12,$4.05,$4.05,$0.20,$33.78,NaN
24195,Hospital Outpatient Department Provider Fee,J0895,PR DEFEROXAMINE MESYLATE INJ,00409-2336-10,$46,$7.20,$46.00,$46.00,$8,$7.78,...,NaN,$8.26,$7.66,$7.66,$8.26,$7.66,$7.66,$7.20,$11.29,NaN
24196,Hospital Outpatient Department Provider Fee,J0897,PR INJ DENOSUMAB 1 MG,55513-0730-01,$30,$19.10,$30.00,$30.00,$23,$23.00,...,NaN,$25.43,$25.01,$25.01,$25.43,$25.01,$25.01,$19.10,$28.12,NaN


In [6]:
def payer_category_from_payer(payer):
    if payer == 'GROSS CHARGE' or payer == 'RECONSTRUCTED CHARGE':
        return 'gross'
    elif payer == 'CASH PRICE':
        return 'cash'
    elif payer == 'DEIDENTIFIED MIN' or payer == 'Min Price':
        return 'min'
    elif payer == 'DEIDENTIFIED MAX' or payer == 'Max Price':
        return 'max'
    
    return 'payer'

In [7]:
df_mid = pd.DataFrame(df_in)

df_mid.columns = list(map(lambda c: c.strip(), df_mid.columns.to_list()))
df_mid = df_mid.rename(columns={
    'Procedure': 'local_code',
    'Code Type': 'line_type',
    'Code': 'code',
    'NDC': 'ndc',
    'CPT/HCPCS Code': 'hcpcs_cpt',
    'Beaumont Health Charge Code Description': 'description',
    'Rev Code': 'rev_code',
    'REV CODE': 'rev_code',
    'Procedure Description': 'description',
    'FS ID': 'additional_generic_notes'
})

if 'GROSS CHARGE' in df_mid.columns.to_list():
    money_idx = df_mid.columns.to_list().index('GROSS CHARGE')
elif 'RECONSTRUCTED CHARGE' in df_mid.columns.to_list():
    money_idx = df_mid.columns.to_list().index('RECONSTRUCTED CHARGE')
    
money_columns = df_mid.columns.to_list()[money_idx:]
remaining_columns = df_mid.columns.to_list()[:money_idx]
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_name', value_name='standard_charge')

df_mid['description'] = df_mid['description'].str.strip()
df_mid['payer_name'] = df_mid['payer_name'].str.strip()
    
df_mid['standard_charge'] = df_mid['standard_charge'].astype(str)
df_mid['standard_charge'] = df_mid['standard_charge'].apply(cleanup_dollar_value)
df_mid = df_mid[df_mid['standard_charge'] != 'nan']
df_mid = df_mid[df_mid['standard_charge'] != '#VALUE!']
df_mid = df_mid[df_mid['standard_charge'] != '#N/A']
df_mid = df_mid[df_mid['standard_charge'] != '-']
df_mid = df_mid[df_mid['standard_charge'] != '!']
df_mid = df_mid.dropna(subset=['standard_charge'], axis=0)
    
df_mid['additional_payer_specific_notes'] = None
df_mid['contracting_method'] = None
df_mid.loc[df_mid['standard_charge'].str.isalpha(), 'contracting_method'] = 'other'
df_mid.loc[df_mid['standard_charge'].str.isalpha(), 'additional_payer_specific_notes'] = df_mid[df_mid['standard_charge'].str.isalpha()]['standard_charge']
df_mid.loc[df_mid['standard_charge'].str.isalpha(), 'standard_charge'] = None
    
if not 'line_type' in df_mid.columns:
    df_mid['line_type'] = None
    
df_mid['eapg'] = None
df_mid['ms_drg'] = None
    
if 'code' in df_mid.columns:
    df_mid.loc[df_mid['line_type'] == 'DRG', 'ms_drg'] = df_mid[df_mid['line_type'] == 'DRG']['code'].apply(
        lambda code: code.split(" ")[-1]
    )
            
    df_mid['code'] = df_mid['code'].fillna('')
    df_mid['code'] = df_mid['code'].apply(lambda code: str(int(code)) if type(code) == int or type(code) == float else code)

    if not 'hcpcs_cpt' in df_mid.columns:
        df_mid['hcpcs_cpt'] = df_mid['code'].apply(lambda code: code if len(code) == 5 else None)
        
    df_mid.loc[df_mid['code'] == '', 'code'] = None
else:
    df_mid['code'] = None
        
if not 'hcpcs_cpt' in df_mid.columns:
    df_mid['hcpcs_cpt'] = None
    
df_mid = df_mid[df_mid['hcpcs_cpt'] != 'deleted']
        
df_mid['payer_category'] = df_mid['payer_name'].apply(payer_category_from_payer)

df_mid['plan_name'] = None

df_mid['hospital_id'] = ccn
df_mid['apr_drg'] = None
df_mid['modifiers'] = None
df_mid['alt_hcpcs_cpt'] = None
df_mid['thru'] = None
df_mid['apc'] = None
df_mid['icd'] = None
df_mid['drug_hcpcs_multiplier'] = None
df_mid['drug_type_of_measurement'] = None
df_mid['drug_quantity'] = None
df_mid['drug_unit_of_measurement'] = None
df_mid['billing_class'] = None
df_mid['standard_charge_percent'] = None

if not 'setting' in df_mid.columns:
    df_mid['setting'] = None

if not 'additional_generic_notes' in df_mid.columns:
    df_mid['additional_generic_notes'] = None

In [8]:
df_mid['eapg'] = df_mid.apply(
    lambda row: None if type(row['eapg']) == str and row['eapg'].startswith('Custom') else row['eapg'], 
    axis=1
)
    
df_mid['hcpcs_cpt'] = df_mid['hcpcs_cpt'].apply(lambda cpt: cpt.replace(" ", "") if type(cpt) == str else cpt)

df_mid['hcpcs_cpt'] = df_mid['hcpcs_cpt'].fillna('')

In [9]:
df_mid[df_mid['hcpcs_cpt'].str.len() > 5][['code', 'description', 'hcpcs_cpt']]

,code,description,hcpcs_cpt
10068,None,CARDIO GENETICS CNSLNG - PATIENT PAY,681139
10069,None,NO SHOW,950003
10070,None,APPT CANCELLED < 24HRS,950008
10071,None,NON CANCEL APPT/NO SHOW MENTOR,950010
10072,None,PH PAPER,950037
...,...,...,...
531977,None,GM EXECUTIVE HEALTH EYE EXAM,951005
531989,None,CAD CHEST CONCURRENT INTERPRET,0174T9
532052,None,"ANOSCOPY,DIAGNOSTIC",46600P
532053,None,CL FIT APHAKIA BILAT,92312P


In [10]:
df_mid[df_mid['hcpcs_cpt'].str.len() == 4][['code', 'description', 'hcpcs_cpt']]

,code,description,hcpcs_cpt
494730,None,"ANESTH,NERVE BLOCKS/INJECTIONS,PRONE",1992
518928,None,"ANESTH,NERVE BLOCKS/INJECTIONS,PRONE",1992


In [11]:
df_mid.loc[df_mid['hcpcs_cpt'].str.len() > 5, 'code'] = df_mid[df_mid['hcpcs_cpt'].str.len() > 5]['hcpcs_cpt']
df_mid.loc[df_mid['hcpcs_cpt'].str.len() > 5, 'hcpcs_cpt'] = None
df_mid.loc[df_mid['hcpcs_cpt'].str.len() == 4, 'code'] = df_mid[df_mid['hcpcs_cpt'].str.len() == 4]['hcpcs_cpt']
df_mid.loc[df_mid['hcpcs_cpt'].str.len() == 4, 'hcpcs_cpt'] = None

In [12]:
df_mid.loc[df_mid['additional_generic_notes'] == 'Office Clinic Provider Fee', 'line_type'] = 'office clinic provider fee'
df_mid.loc[df_mid['additional_generic_notes'] == 'Hospital Outpatient Department Provider Fee', 'line_type'] = 'hosp. OP dept. provider fee'

df_mid.loc[df_mid['hcpcs_cpt'] == 'RN001', 'code'] = 'RN001'
df_mid.loc[df_mid['hcpcs_cpt'] == 'RN001', 'hcpcs_cpt'] = None

df_mid.loc[df_mid['hcpcs_cpt'] == '', 'hcpcs_cpt'] = None

if 'local_code' in df_mid.columns:
    df_mid['local_code'] = df_mid['local_code'].str[:40]
else:
    df_mid['local_code'] = None

if not 'rev_code' in df_mid.columns:
    df_mid['rev_code'] = None

df_mid['ndc'] = df_mid['ndc'].fillna('')
df_mid.loc[df_mid['ndc'].str.startswith("WBH"), 'ndc'] = None
df_mid.loc[df_mid['ndc'] == '', 'ndc'] = None

In [13]:
df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])
df_out

,hospital_id,line_type,description,rev_code,local_code,code,ms_drg,apr_drg,eapg,hcpcs_cpt,...,billing_class,setting,payer_category,payer_name,plan_name,standard_charge,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
0,230020,office clinic provider fee,PR FINE NEEDLE ASPIRATION BX W/CT GDN 1ST LESION,None,None,None,None,None,None,10009,...,None,None,gross,GROSS CHARGE,None,618.00,None,None,Office Clinic Provider Fee,None
1,230020,office clinic provider fee,PR FINE NEEDLE ASPIRATION BX W/O IMG GDN 1ST L...,None,None,None,None,None,None,10021,...,None,None,gross,GROSS CHARGE,None,184.00,None,None,Office Clinic Provider Fee,None
2,230020,office clinic provider fee,FNA W/O IMAGING GUIDANCE,None,None,None,None,None,None,10021,...,None,None,gross,GROSS CHARGE,None,184.00,None,None,Office Clinic Provider Fee,None
3,230020,office clinic provider fee,PR IMAGE-GUIDED CATHETER FLUID COLLECTION DRAI...,None,None,None,None,None,None,10030,...,None,None,gross,GROSS CHARGE,None,443.00,None,None,Office Clinic Provider Fee,None
4,230020,office clinic provider fee,GUIDE CATH FLUID DRAINAGE,None,None,None,None,None,None,10030,...,None,None,gross,GROSS CHARGE,None,443.00,None,None,Office Clinic Provider Fee,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532351,230020,hosp. OP dept. provider fee,"EPOETIN ALFA, NON-ESRD",None,None,None,None,None,None,J0885,...,None,None,payer,Max Price,None,10.90,None,None,Hospital Outpatient Department Provider Fee,None
532352,230020,hosp. OP dept. provider fee,PR DECITABINE INJECTION,None,None,None,None,None,None,J0894,...,None,None,payer,Max Price,None,33.78,None,None,Hospital Outpatient Department Provider Fee,None
532353,230020,hosp. OP dept. provider fee,PR DEFEROXAMINE MESYLATE INJ,None,None,None,None,None,None,J0895,...,None,None,payer,Max Price,None,11.29,None,None,Hospital Outpatient Department Provider Fee,None
532354,230020,hosp. OP dept. provider fee,PR INJ DENOSUMAB 1 MG,None,None,None,None,None,None,J0897,...,None,None,payer,Max Price,None,28.12,None,None,Hospital Outpatient Department Provider Fee,None


In [14]:
df_out.to_csv("rate_" + ccn + ".csv", index=False)